In [1]:
import pandas as pd
import vectorbtpro as vbt


/Users/ericervin/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [2]:
vbt.settings.plotting["layout"]["template"] = "vbt_dark"
vbt.settings.plotting["layout"]["width"] = 1200
vbt.settings.plotting['layout']['height'] = 200
vbt.settings.wrapping["freq"] = "1m"
# vbt.settings.portfolio['size_granularity'] = 0.001
vbt.settings.portfolio['init_cash'] = 10000

In [3]:
data_path = '/Users/ericervin/Documents/Coding/data-repository/data/BTCUSDT_1m_futures.pkl' 

futures_1m = vbt.BinanceData.load(data_path)

In [4]:


def dollar_bar_func(ohlc_df, dollar_bar_size):
    # Calculate dollar value traded for each row
    ohlc_df['DollarValue'] = ohlc_df['Close'] * ohlc_df['Volume']
    
    # Calculate cumulative dollar value
    ohlc_df['CumulativeDollarValue'] = ohlc_df['DollarValue'].cumsum()
    
    # Determine the number of dollar bars
    num_bars = int(ohlc_df['CumulativeDollarValue'].iloc[-1] / dollar_bar_size)
    
    # Generate index positions for dollar bars
    bar_indices = [0]
    cumulative_value = 0
    for i in range(1, len(ohlc_df)):
        cumulative_value += ohlc_df['DollarValue'].iloc[i]
        if cumulative_value >= dollar_bar_size:
            bar_indices.append(i)
            cumulative_value = 0
    
    # Create a new dataframe with dollar bars
    dollar_bars = []
    for i in range(len(bar_indices) - 1):
        start_idx = bar_indices[i]
        end_idx = bar_indices[i + 1]
        
        dollar_bar = {
            'Open': ohlc_df['Open'].iloc[start_idx],
            'High': ohlc_df['High'].iloc[start_idx:end_idx].max(),
            'Low': ohlc_df['Low'].iloc[start_idx:end_idx].min(),
            'Close': ohlc_df['Close'].iloc[end_idx],
            'Volume': ohlc_df['Volume'].iloc[start_idx:end_idx].sum(),
            'Quote volume': ohlc_df['Quote volume'].iloc[start_idx:end_idx].sum(),
            'Trade count': ohlc_df['Trade count'].iloc[start_idx:end_idx].sum(),
            'Taker base volume': ohlc_df['Taker base volume'].iloc[start_idx:end_idx].sum(),
            'Taker quote volume': ohlc_df['Taker quote volume'].iloc[start_idx:end_idx].sum()
        }
        
        if isinstance(ohlc_df.index, pd.DatetimeIndex):
            dollar_bar['Open Time'] = ohlc_df.index[start_idx]
            dollar_bar['Close Time'] = ohlc_df.index[end_idx] - pd.Timedelta(milliseconds=1)
        elif 'Open Time' in ohlc_df.columns:
            dollar_bar['Open Time'] = ohlc_df['Open Time'].iloc[start_idx]
            dollar_bar['Close Time'] = ohlc_df['Open Time'].iloc[end_idx] - pd.Timedelta(milliseconds=1)
        
        dollar_bars.append(dollar_bar)
    
    dollar_bars_df = pd.concat([pd.DataFrame([bar]) for bar in dollar_bars], ignore_index=True)
    
    return dollar_bars_df

# Create a simple function to simplify the number so we can use it in our column names
def simplify_number(num):
    """
    Simplifies a large number by converting it to a shorter representation with a suffix (K, M, B).
    simplify_number(1000) -> 1K
    """
    suffixes = ['', 'K', 'M', 'B']
    suffix_index = 0

    while abs(num) >= 1000 and suffix_index < len(suffixes) - 1:
        num /= 1000.0
        suffix_index += 1

    suffix = suffixes[suffix_index] if suffix_index > 0 else ''
    simplified_num = f'{int(num)}{suffix}'

    return simplified_num

def merge_and_fill_dollar_bars(original_df, dollar_bars_df, dollar_bar_size):
    # Add prefix to column names in dollar bars dataframe
    dollar_bar_prefix = f'db_{simplify_number(dollar_bar_size)}_'
    dollar_bars_df_renamed = dollar_bars_df.add_prefix(dollar_bar_prefix)

    # Convert 'Open Time' columns to pandas datetime format and set them as index
    dollar_bars_df_renamed.index = pd.to_datetime(dollar_bars_df_renamed[dollar_bar_prefix + 'Open Time'])

    # Merge the dataframes on the index
    merged_df = original_df.merge(dollar_bars_df_renamed, how='left', left_index=True, right_index=True)

    # Set the flag for a new dollar bar with prefix
    merged_df[dollar_bar_prefix + 'NewDBFlag'] = ~merged_df[dollar_bar_prefix + 'Close'].isna()

    # Forward fill the NaN values for all columns except the new dollar bar flag
    columns_to_ffill = [col for col in merged_df.columns if col != dollar_bar_prefix + 'NewDBFlag']
    merged_df[columns_to_ffill] = merged_df[columns_to_ffill].fillna(method='ffill')

    # Fill the remaining NaN values in the new dollar bar flag column with False
    merged_df[dollar_bar_prefix + 'NewDBFlag'] = merged_df[dollar_bar_prefix + 'NewDBFlag'].fillna(False)
    
    # Assign the renamed 'Open Time' column back to the dataframe
    merged_df[dollar_bar_prefix + 'Open Time'] = merged_df[dollar_bar_prefix + 'Open Time']

    return merged_df





In [7]:
# Rather than make new dollar bars just read in the pkl file
dollar_bar_size = 90_000_000
dollar_bars_90M = vbt.BinanceData.load('data/btc_90M_db_vbt.pkl')
dollar_bar_copy = dollar_bars_90M.get()
futures_1m_copy = futures_1m.get()

## Read in the Predictions from the Model
Read in the predictions and attach them to the dollar bar dataframe

In [11]:
from joblib import load, dump
insample_model = load('models/model_upto_2023.joblib')


In [ ]:
filename = 'models/model_upto_2023.joblib'
# Load the model from the .joblib file
final_pipeline = load(filename) 

# Make predictions on the entire dataset
predictions = final_pipeline.predict(X)

# Calculate the R-squared score on the entire dataset
r2 = r2_score(y, predictions)

print(f"R-squared on the entire dataset: {r2}")

In [9]:
# Create a new dataframe with the dollar bars merged into the original dataframe
full_df = merge_and_fill_dollar_bars(futures_1m_copy, dollar_bar_copy, dollar_bar_size)

In [10]:
full_df

,Open,High,Low,Close,Volume,Quote volume,Trade count,Taker base volume,Taker quote volume,db_90M_Open,...,db_90M_Low,db_90M_Close,db_90M_Volume,db_90M_Quote volume,db_90M_Trade count,db_90M_Taker base volume,db_90M_Taker quote volume,db_90M_Open Time,db_90M_Close Time,db_90M_NewDBFlag
Open time,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,28948.19,28997.16,28935.30,28991.01,143.661,4.161510e+06,1120,114.439,3.315299e+06,28948.19,...,28706.0,28796.3,3181.960,9.183460e+07,24166.0,1277.389,3.689901e+07,2021-01-01 00:00:00+00:00,2021-01-01 00:14:59.999000+00:00,True
2021-01-01 00:01:00+00:00,28992.98,29045.93,28991.01,29035.18,256.280,7.437731e+06,1967,179.650,5.213713e+06,28948.19,...,28706.0,28796.3,3181.960,9.183460e+07,24166.0,1277.389,3.689901e+07,2021-01-01 00:00:00+00:00,2021-01-01 00:14:59.999000+00:00,False
2021-01-01 00:02:00+00:00,29036.41,29036.97,28993.19,29016.23,102.675,2.979644e+06,987,44.060,1.278719e+06,28948.19,...,28706.0,28796.3,3181.960,9.183460e+07,24166.0,1277.389,3.689901e+07,2021-01-01 00:00:00+00:00,2021-01-01 00:14:59.999000+00:00,False
2021-01-01 00:03:00+00:00,29016.23,29023.87,28995.50,29002.92,85.762,2.487845e+06,832,56.768,1.646753e+06,28948.19,...,28706.0,28796.3,3181.960,9.183460e+07,24166.0,1277.389,3.689901e+07,2021-01-01 00:00:00+00:00,2021-01-01 00:14:59.999000+00:00,False
2021-01-01 00:04:00+00:00,29005.93,29019.98,29000.00,29000.00,70.246,2.037999e+06,752,46.645,1.353346e+06,28948.19,...,28706.0,28796.3,3181.960,9.183460e+07,24166.0,1277.389,3.689901e+07,2021-01-01 00:00:00+00:00,2021-01-01 00:14:59.999000+00:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-03 23:08:00+00:00,27055.50,27058.10,27055.50,27058.10,33.934,9.181214e+05,538,10.977,2.970001e+05,27036.20,...,27010.0,27050.8,3446.435,9.321961e+07,35797.0,1852.779,5.012026e+07,2023-06-03 22:22:00+00:00,2023-06-03 22:55:59.999000+00:00,False
2023-06-03 23:09:00+00:00,27058.10,27060.00,27058.00,27060.00,22.295,6.032967e+05,375,18.930,5.122406e+05,27036.20,...,27010.0,27050.8,3446.435,9.321961e+07,35797.0,1852.779,5.012026e+07,2023-06-03 22:22:00+00:00,2023-06-03 22:55:59.999000+00:00,False
2023-06-03 23:10:00+00:00,27060.00,27060.50,27059.90,27060.50,40.601,1.098664e+06,494,24.383,6.598049e+05,27036.20,...,27010.0,27050.8,3446.435,9.321961e+07,35797.0,1852.779,5.012026e+07,2023-06-03 22:22:00+00:00,2023-06-03 22:55:59.999000+00:00,False
